In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv("covid_toy.csv")

In [5]:
df.sample(10)

,age,gender,fever,cough,city,has_covid
66,51,Male,104.0,Mild,Kolkata,No
99,10,Female,98.0,Strong,Kolkata,Yes
29,34,Female,NaN,Strong,Mumbai,Yes
11,65,Female,98.0,Mild,Mumbai,Yes
37,55,Male,100.0,Mild,Kolkata,No
22,71,Female,98.0,Strong,Kolkata,Yes
56,71,Male,NaN,Strong,Kolkata,No
81,65,Male,99.0,Mild,Delhi,No
60,24,Female,102.0,Strong,Bangalore,Yes
57,49,Female,99.0,Strong,Bangalore,No


In [6]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['has_covid']), df['has_covid'],
                                                   test_size = 0.2)

In [8]:
X_train

,age,gender,fever,cough,city
42,27,Male,100.0,Mild,Delhi
37,55,Male,100.0,Mild,Kolkata
48,66,Male,99.0,Strong,Bangalore
45,72,Male,99.0,Mild,Bangalore
41,82,Male,NaN,Mild,Kolkata
...,...,...,...,...,...
98,5,Female,98.0,Strong,Mumbai
40,49,Female,102.0,Mild,Delhi
30,15,Male,101.0,Mild,Delhi
65,69,Female,102.0,Mild,Bangalore


In [10]:
X_test.head()

,age,gender,fever,cough,city
88,5,Female,100.0,Mild,Kolkata
80,14,Female,99.0,Mild,Mumbai
99,10,Female,98.0,Strong,Kolkata
63,10,Male,100.0,Mild,Bangalore
28,16,Male,104.0,Mild,Kolkata


# 1. Without Column Transformer

In [14]:
# adding simple imputer to fever column
# simple imputer fill the missing values with mean value from the given column i.e fever
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [15]:
# ordinal encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', "Strong"]])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [18]:
# one hot encoder -> gender, city
ohe = OneHotEncoder(drop='first', sparse_output = False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

X_train_gender_city.shape

(80, 4)

In [20]:
# Extracting age
X_train_age = X_train.drop(columns = ['gender', 'fever', 'cough', 'city']).values

# also the test data
X_test_age = X_test.drop(columns = ['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [24]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis = 1)

# also for test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis = 1)

X_train_transformed.shape

(80, 7)

In [30]:
X_train_transformed

array([[ 27.        , 100.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 55.        , 100.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 66.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 72.        ,  99.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 82.        , 101.07042254,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 42.        , 101.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 20.        , 101.07042254,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [  6.        , 104.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 48.        , 103.        ,   0.        ,   0.        ,
          1.    

# 2. Using Column Transformer

In [25]:
from sklearn.compose import ColumnTransformer

In [27]:
transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output = False, drop = 'first'), ['gender', 'city'])
], remainder = 'passthrough')

In [29]:
transformer.fit_transform(X_train)

array([[100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  55.        ],
       [ 99.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  66.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  72.        ],
       [101.07042254,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  82.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [101.07042254,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   6.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.    

In [31]:
transformer.fit_transform(X_train).shape

(80, 7)

In [32]:
transformer.transform(X_test)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  14.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  10.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  16.        ],
       [102.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  33.        ],
       [ 99.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  65.        ],
       [ 98.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  83.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.    

In [33]:
transformer.transform(X_test).shape

(20, 7)